In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="workspace")
exp = Experiment(workspace=ws, name="workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cpu_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace = ws, name=cpu_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
    - azureml-defaults

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.01, 100),
    "--max-iter": choice(1, 10, 100, 1000, 10000)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=None, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', './training')
# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory='./training',
                    script='train.py',
                    compute_target=compute_target,
                    environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd 

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x,y)

training = pd.concat([x_train, y_train])
test = pd.concat([x_test, y_test])

if not os.path.isdir('data'):
    os.mkdir('data')

pd.DataFrame(training).to_csv('data/train.csv', index=False)
pd.DataFrame(test).to_csv('data/test.csv', index=False)

ds_def = ws.get_default_datastore()
ds_def.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)

training_ds = TabularDatasetFactory.from_delimited_files(path=ds_def.path('bankmarketing/train.csv'))
test_ds = TabularDatasetFactory.from_delimited_files(path=ds_def.path('bankmarketing/test.csv'))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_ds,
    label_column_name='y',
    n_cross_validations=2,
    compute_target=cpu_cluster)

In [2]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=False)

In [ ]:
automl_run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

from azureml.core.run import Run 

print(automl_run.id)
model_explained = Run(experiment=exp, runid=automl_run.id)
model_explained.wait_for_completion()

best_run, fitted_model = automl_run.get_output()
print(best_run)

joblib.dump(fitted_model, 'automl.pkl', compress=1)
